In [1]:
import pickle as pkl
import numpy as np
import pandas as pd

from zipfile import ZipFile
from sklearn.metrics import accuracy_score, matthews_corrcoef, confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.backend import clear_session

from twitter_nlp_toolkit.tweet_sentiment_classifier import tweet_sentiment_classifier
from twitter_nlp_toolkit.file_fetcher import file_fetcher



In [2]:
# Install Spacy library
!python -m spacy download en_core_web_sm

[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [3]:
clear_session()

In [4]:
chunk = 1 # Fraction of data to train on - you can reduce for debugging for speed
model_path = '.models'
redownload=True

Here we download pre-trained models and a validation dataset. The models have been pre-trained on the Sentiment140 dataset, taken form here: https://www.kaggle.com/kazanova/sentiment140

The validation data is hand-labeled airline customer feedback taken from https://www.figure-eight.com/data-for-everyone/

In [5]:
if redownload: 
    
    # Validation data
    file_fetcher.download_file('https://www.dropbox.com/s/muqov03s9vtbe92/tweets_airline.zip?dl=1',"tweets_airline.zip")
    


tweets_airline.zip: 100%|#########################################################| 1.07M/1.07M [00:00<00:00, 3.78MB/s]


In [6]:
# Load the validation data

test_data = pd.read_csv('tweets_airline.zip', header=0,
                        names=['Index', 'Sentiment', 'Sentiment_confidence',
                                'Negative_reason', 'Negative_reason_confidence',
                                 'Airline', 'Airline_sentiment_gold', 'Handle',
                                 'Negative_reason_gold', 'Retweet_count', 'Text',
                                'Tweet_coord', 'Time', 'Location', 'Timezone'])

In [7]:
test_data.head(5)

,Index,Sentiment,Sentiment_confidence,Negative_reason,Negative_reason_confidence,Airline,Airline_sentiment_gold,Handle,Negative_reason_gold,Retweet_count,Text,Tweet_coord,Time,Location,Timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


We drop the tweets with "neural" sentiment:

In [8]:
# Remove the unlabeled test data

test_data['Labels'] = (test_data['Sentiment'] == 'positive') * 2
test_data['Labels'] = test_data['Labels'] + (test_data['Sentiment'] == 'neutral') * 1
test_data['Labels'] = test_data['Labels'] / 2

test_data.set_index('Labels')
test_data = test_data[test_data.Labels != 0.5]

In [9]:
# Download the small ensemble
# Executing this cell starts a 500MB download

Classifier = tweet_sentiment_classifier.SentimentAnalyzer()
Classifier.load_small_ensemble()

bow_1Mtweets
Default parameters file not found or not working...
Trying .models/small_ensemble/bow_1Mtweets
Loading BoW model bow_1Mtweets from legacy parameter file
BoW model bow_1Mtweets loaded successfully
glove_100D_10N_340000V_719685T_Twitter_0
Default parameters file not found or not working...
Trying .models/small_ensemble/glove_100D_10N_340000V_719685T_Twitter_0
Loading GloVE model glove_100D_10N_340000V_719685T_Twitter_0 from legacy parameter file
Pre-trained embedding model glove_100D_10N_340000V_719685T_Twitter_0 loaded successfully
glove_200D_10N_20000V_343332T_Twitter_7
Default parameters file not found or not working...
Trying .models/small_ensemble/glove_200D_10N_20000V_343332T_Twitter_7
Loading GloVE model glove_200D_10N_20000V_343332T_Twitter_7 from legacy parameter file
Pre-trained embedding model glove_200D_10N_20000V_343332T_Twitter_7 loaded successfully


We sanity check the models: 

In [10]:
Classifier.predict(['I am happy', 'I am sad', 'I am cheerful', 'I am mad'])


array([1., 0., 1., 0.])

We test the model on an airline customer feedback dataset.

In [11]:
# Executing this cell takes several minuites on a laptop

predictions = Classifier.predict(test_data['Text'])


In [12]:
print('Test Accuracy:  {:.3f}'.format(accuracy_score(test_data['Labels'], predictions)))
print('Test MCC:  {:.3f}'.format(matthews_corrcoef(test_data['Labels'], predictions)))
confusion_matrix(test_data['Labels'], predictions)

Test Accuracy:  0.812
Test MCC:  0.570


array([[7305, 1873],
       [ 297, 2066]], dtype=int64)

We have accuracy of just over 80%.

We split our evaluation dataset into validation and testing and check for poor-performing models:

In [13]:
valX, testX, valY, testY = train_test_split(test_data['Text'], test_data['Labels'], test_size=0.5, stratify=test_data['Labels'])

In [14]:
# Executing this cell takes several minuites on a laptop

Classifier.trim_models(valX, valY, threshold=0.7)

Model bow_1Mtweets score: 0.795
Model glove_100D_10N_340000V_719685T_Twitter_0 score: 0.788
Model glove_200D_10N_20000V_343332T_Twitter_7 score: 0.786


All three models perform well, so none have been pruned

In [15]:
predictions = Classifier.predict(testX)

print('Test Accuracy:  {:.3f}'.format(accuracy_score(testY, predictions)))
print('Test MCC:  {:.3f}'.format(matthews_corrcoef(testY, predictions)))
confusion_matrix(testY, predictions)

Test Accuracy:  0.818
Test MCC:  0.584


array([[3670,  919],
       [ 134, 1048]], dtype=int64)

To improve our accuracy, we can refine the models on our airline data. The early stopping procedure (enabled by default to use 20% of the training data for validation) should minimize overfitting.

In [16]:
Classifier.refine(valX, valY)

 Preprocessed 5770 tweets 

 Filtered data
 Preprocessed 5770 tweets 
Filtered data
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 4616 samples, validate on 1154 samples
Epoch 1/500
4616/4616 [==============================] - 4s 817us/sample - loss: 0.5021 - acc: 0.7695 - val_loss: 0.4576 - val_acc: 0.7998
Epoch 2/500
4616/4616 [==============================] - 0s 25us/sample - loss: 0.4872 - acc: 0.7727 - val_loss: 0.4444 - val_acc: 0.8102
Epoch 3/500
4616/4616 [==============================] - 0s 25us/sample - loss: 0.4747 - acc: 0.7938 - val_loss: 0.4319 - val_acc: 0.8206
Epoch 4/500
4616/4616 [==============================] - 0s 26us/sample - loss: 0.4640 - acc: 0.7979 - val_loss: 0.4200 - val_acc: 0.8319
Epoch 5/500
4616/4616 [==============================] - 0s 25us/sample - loss: 0.4554 - acc: 0.8061 - val_loss: 0.4089 - val_acc: 0.8380
Epoch 6/500
4616/4616 [==============================] - 0s 25us/sample - loss: 0.4475 - acc: 0.8200 - val_loss: 0.3985 - val_a

Epoch 58/500
4616/4616 [==============================] - 0s 26us/sample - loss: 0.3215 - acc: 0.8871 - val_loss: 0.2826 - val_acc: 0.8951
Epoch 59/500
4616/4616 [==============================] - 0s 25us/sample - loss: 0.3073 - acc: 0.8889 - val_loss: 0.2821 - val_acc: 0.8943
Epoch 60/500
4616/4616 [==============================] - 0s 25us/sample - loss: 0.3177 - acc: 0.8915 - val_loss: 0.2816 - val_acc: 0.8943
Epoch 61/500
4616/4616 [==============================] - 0s 26us/sample - loss: 0.3077 - acc: 0.8923 - val_loss: 0.2811 - val_acc: 0.8943
Epoch 62/500
4616/4616 [==============================] - 0s 25us/sample - loss: 0.3199 - acc: 0.8858 - val_loss: 0.2807 - val_acc: 0.8943
Epoch 63/500
4616/4616 [==============================] - 0s 25us/sample - loss: 0.3072 - acc: 0.8915 - val_loss: 0.2802 - val_acc: 0.8943
Epoch 64/500
4616/4616 [==============================] - 0s 25us/sample - loss: 0.3063 - acc: 0.8906 - val_loss: 0.2796 - val_acc: 0.8934
Epoch 65/500
4616/4616 [===

Epoch 117/500
4616/4616 [==============================] - 0s 26us/sample - loss: 0.2887 - acc: 0.8988 - val_loss: 0.2457 - val_acc: 0.9012
Epoch 118/500
4616/4616 [==============================] - 0s 27us/sample - loss: 0.2786 - acc: 0.9032 - val_loss: 0.2457 - val_acc: 0.9012
Epoch 119/500
4616/4616 [==============================] - 0s 26us/sample - loss: 0.2782 - acc: 0.9003 - val_loss: 0.2456 - val_acc: 0.9012
Epoch 120/500
4616/4616 [==============================] - 0s 26us/sample - loss: 0.2688 - acc: 0.9014 - val_loss: 0.2456 - val_acc: 0.9021
Epoch 121/500
4616/4616 [==============================] - 0s 26us/sample - loss: 0.2687 - acc: 0.9040 - val_loss: 0.2455 - val_acc: 0.9029
Epoch 122/500
4616/4616 [==============================] - 0s 31us/sample - loss: 0.2775 - acc: 0.8980 - val_loss: 0.2454 - val_acc: 0.9029
Epoch 123/500
4616/4616 [==============================] - 0s 26us/sample - loss: 0.2809 - acc: 0.8958 - val_loss: 0.2452 - val_acc: 0.9029
Epoch 124/500
4616/4

Epoch 176/500
4616/4616 [==============================] - 0s 26us/sample - loss: 0.2650 - acc: 0.9027 - val_loss: 0.2384 - val_acc: 0.9099
Epoch 177/500
4616/4616 [==============================] - 0s 29us/sample - loss: 0.2646 - acc: 0.9055 - val_loss: 0.2394 - val_acc: 0.9099
Epoch 178/500
4616/4616 [==============================] - 0s 26us/sample - loss: 0.2598 - acc: 0.9086 - val_loss: 0.2403 - val_acc: 0.9099
Epoch 179/500
4616/4616 [==============================] - 0s 26us/sample - loss: 0.2594 - acc: 0.9064 - val_loss: 0.2411 - val_acc: 0.9081
Epoch 180/500
4616/4616 [==============================] - 0s 28us/sample - loss: 0.2675 - acc: 0.9001 - val_loss: 0.2418 - val_acc: 0.9064
Epoch 181/500
4616/4616 [==============================] - 0s 27us/sample - loss: 0.2635 - acc: 0.9068 - val_loss: 0.2425 - val_acc: 0.9055
Epoch 182/500
4616/4616 [==============================] - 0s 26us/sample - loss: 0.2713 - acc: 0.9029 - val_loss: 0.2430 - val_acc: 0.9064
Epoch 183/500
4616/4

Epoch 11/500
4616/4616 [==============================] - 0s 34us/sample - loss: 0.3693 - acc: 0.8588 - val_loss: 0.3326 - val_acc: 0.8718
Epoch 12/500
4616/4616 [==============================] - 0s 34us/sample - loss: 0.3637 - acc: 0.8689 - val_loss: 0.3250 - val_acc: 0.8709
Epoch 13/500
4616/4616 [==============================] - 0s 33us/sample - loss: 0.3645 - acc: 0.8657 - val_loss: 0.3181 - val_acc: 0.8726
Epoch 14/500
4616/4616 [==============================] - 0s 34us/sample - loss: 0.3424 - acc: 0.8815 - val_loss: 0.3120 - val_acc: 0.8804
Epoch 15/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.3384 - acc: 0.8744 - val_loss: 0.3063 - val_acc: 0.8821
Epoch 16/500
4616/4616 [==============================] - 0s 33us/sample - loss: 0.3364 - acc: 0.8761 - val_loss: 0.3013 - val_acc: 0.8873
Epoch 17/500
4616/4616 [==============================] - 0s 35us/sample - loss: 0.3468 - acc: 0.8748 - val_loss: 0.2975 - val_acc: 0.8891
Epoch 18/500
4616/4616 [===

...

Epoch 129/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.2686 - acc: 0.9071 - val_loss: 0.2507 - val_acc: 0.9064
Epoch 130/500
4616/4616 [==============================] - 0s 34us/sample - loss: 0.2553 - acc: 0.9066 - val_loss: 0.2505 - val_acc: 0.9064
Epoch 131/500
4616/4616 [==============================] - 0s 33us/sample - loss: 0.2479 - acc: 0.9053 - val_loss: 0.2502 - val_acc: 0.9073
Epoch 132/500
4616/4616 [==============================] - 0s 36us/sample - loss: 0.2556 - acc: 0.9066 - val_loss: 0.2500 - val_acc: 0.9081
Epoch 133/500
4616/4616 [==============================] - 0s 35us/sample - loss: 0.2478 - acc: 0.9073 - val_loss: 0.2498 - val_acc: 0.9081
Epoch 134/500
4616/4616 [==============================] - 0s 35us/sample - loss: 0.2507 - acc: 0.9038 - val_loss: 0.2496 - val_acc: 0.9073
Epoch 135/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.2540 - acc: 0.9094 - val_loss: 0.2494 - val_acc: 0.9073
Epoch 136/500
4616/4

Epoch 188/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.2411 - acc: 0.9127 - val_loss: 0.2441 - val_acc: 0.9107
Epoch 189/500
4616/4616 [==============================] - 0s 33us/sample - loss: 0.2368 - acc: 0.9114 - val_loss: 0.2440 - val_acc: 0.9116
Epoch 190/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.2468 - acc: 0.9114 - val_loss: 0.2440 - val_acc: 0.9116
Epoch 191/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.2452 - acc: 0.9092 - val_loss: 0.2439 - val_acc: 0.9107
Epoch 192/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.2413 - acc: 0.9120 - val_loss: 0.2438 - val_acc: 0.9107
Epoch 193/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.2434 - acc: 0.9129 - val_loss: 0.2438 - val_acc: 0.9107
Epoch 194/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.2399 - acc: 0.9081 - val_loss: 0.2437 - val_acc: 0.9107
Epoch 195/500
4616/4

Epoch 247/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.2350 - acc: 0.9105 - val_loss: 0.2405 - val_acc: 0.9125
Epoch 248/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.2393 - acc: 0.9107 - val_loss: 0.2404 - val_acc: 0.9125
Epoch 249/500
4616/4616 [==============================] - 0s 34us/sample - loss: 0.2353 - acc: 0.9123 - val_loss: 0.2403 - val_acc: 0.9133
Epoch 250/500
4616/4616 [==============================] - 0s 34us/sample - loss: 0.2311 - acc: 0.9146 - val_loss: 0.2401 - val_acc: 0.9133
Epoch 251/500
4616/4616 [==============================] - 0s 33us/sample - loss: 0.2338 - acc: 0.9140 - val_loss: 0.2400 - val_acc: 0.9133
Epoch 252/500
4616/4616 [==============================] - 0s 33us/sample - loss: 0.2396 - acc: 0.9084 - val_loss: 0.2398 - val_acc: 0.9125
Epoch 253/500
4616/4616 [==============================] - 0s 33us/sample - loss: 0.2293 - acc: 0.9153 - val_loss: 0.2397 - val_acc: 0.9125
Epoch 254/500
4616/4

Epoch 306/500
4616/4616 [==============================] - 0s 33us/sample - loss: 0.2408 - acc: 0.9175 - val_loss: 0.2366 - val_acc: 0.9116
Epoch 307/500
4616/4616 [==============================] - 0s 34us/sample - loss: 0.2309 - acc: 0.9114 - val_loss: 0.2366 - val_acc: 0.9116
Epoch 308/500
4616/4616 [==============================] - 0s 33us/sample - loss: 0.2262 - acc: 0.9146 - val_loss: 0.2367 - val_acc: 0.9116
Epoch 309/500
4616/4616 [==============================] - 0s 33us/sample - loss: 0.2274 - acc: 0.9144 - val_loss: 0.2368 - val_acc: 0.9116
Epoch 310/500
4616/4616 [==============================] - 0s 33us/sample - loss: 0.2200 - acc: 0.9155 - val_loss: 0.2370 - val_acc: 0.9116
Epoch 311/500
4616/4616 [==============================] - 0s 35us/sample - loss: 0.2238 - acc: 0.9166 - val_loss: 0.2372 - val_acc: 0.9116
Epoch 312/500
4616/4616 [==============================] - 0s 33us/sample - loss: 0.2247 - acc: 0.9179 - val_loss: 0.2375 - val_acc: 0.9116
Epoch 313/500
4616/4

Epoch 365/500
4616/4616 [==============================] - 0s 32us/sample - loss: 0.2276 - acc: 0.9116 - val_loss: 0.2327 - val_acc: 0.9116
Epoch 366/500
4616/4616 [==============================] - 0s 34us/sample - loss: 0.2281 - acc: 0.9168 - val_loss: 0.2327 - val_acc: 0.9116
Epoch 367/500
4616/4616 [==============================] - 0s 36us/sample - loss: 0.2231 - acc: 0.9168 - val_loss: 0.2327 - val_acc: 0.9116
Epoch 368/500
4616/4616 [==============================] - 0s 35us/sample - loss: 0.2229 - acc: 0.9142 - val_loss: 0.2327 - val_acc: 0.9116
Epoch 369/500
4616/4616 [==============================] - 0s 34us/sample - loss: 0.2247 - acc: 0.9177 - val_loss: 0.2327 - val_acc: 0.9116
Epoch 370/500
4616/4616 [==============================] - 0s 34us/sample - loss: 0.2180 - acc: 0.9183 - val_loss: 0.2328 - val_acc: 0.9116
Epoch 371/500
4616/4616 [==============================] - 0s 33us/sample - loss: 0.2205 - acc: 0.9164 - val_loss: 0.2329 - val_acc: 0.9116
Epoch 372/500
4616/4

In [17]:
test_predictions = Classifier.predict(testX)

print('Test Accuracy:  {:.3f}'.format(accuracy_score(testY, test_predictions)))
print('Test MCC:  {:.3f}'.format(matthews_corrcoef(testY, test_predictions)))
confusion_matrix(testY, predictions)

Test Accuracy:  0.915
Test MCC:  0.723


array([[3670,  919],
       [ 134, 1048]], dtype=int64)

Now we have accuracies of over 90%! 

In [18]:
Classifier.evaluate(testX, testY)

Model bow_1Mtweets accuracy_score: 0.887
Model glove_100D_10N_340000V_719685T_Twitter_0 accuracy_score: 0.912
Model glove_200D_10N_20000V_343332T_Twitter_7 accuracy_score: 0.919


{'bow_1Mtweets': 0.8866747530757234,
 'glove_100D_10N_340000V_719685T_Twitter_0': 0.9116271010223531,
 'glove_200D_10N_20000V_343332T_Twitter_7': 0.9189048691734535,
 'ensembled': 0.9149194247097556}